In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split, Subset
import numpy as np
import os


In [ ]:

DATASET_PATH = "/content/cat_dataset"
NUM_CLASSES = 40
IMAGE_SIZE = 224
BATCH_SIZE = 8
EPOCHS = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **DATASET & TRANSFORM**

In [ ]:

transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

dataset = datasets.ImageFolder(DATASET_PATH, transform=transform)
class_names = dataset.classes


# **TRAIN / VAL / TEST SPLIT**

In [ ]:


train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_data, val_data, test_data = random_split(
    dataset, [train_size, val_size, test_size]
)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


# **PRETRAINED RESNET-50**

In [ ]:


resnet50 = models.resnet50(pretrained=True)

for param in resnet50.parameters():
    param.requires_grad = False

resnet50.fc = nn.Linear(resnet50.fc.in_features, NUM_CLASSES)
resnet50 = resnet50.to(device)


# **CUSTOM CNN MODEL**

In [ ]:

class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# **TRAINING FUNCTION**

In [ ]:


criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, val_loader, optimizer, epochs):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)

        model.eval()
        val_correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_correct += (outputs.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


# **EVALUATION FUNCTION**

In [ ]:

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            correct += (outputs.argmax(1) == labels).sum().item()
    return correct / len(test_loader.dataset)


# **FEW-SHOT LEARNING (2-SHOT)**

In [ ]:

few_shot_indices = []
shots = 2
count = {i: 0 for i in range(NUM_CLASSES)}

for idx, (_, label) in enumerate(dataset):
    if count[label] < shots:
        few_shot_indices.append(idx)
        count[label] += 1

few_shot_dataset = Subset(dataset, few_shot_indices)
few_shot_loader = DataLoader(few_shot_dataset, batch_size=4, shuffle=True)

few_shot_model = CustomCNN(NUM_CLASSES)
optimizer_few = optim.Adam(few_shot_model.parameters(), lr=0.0001)

train_model(few_shot_model, few_shot_loader, val_loader, optimizer_few, epochs=5)


# **ZERO-SHOT LEARNING**

In [ ]:
import clip
from PIL import Image

clip_model, preprocess = clip.load("ViT-B/32", device=device)

text_inputs = torch.cat([
    clip.tokenize(f"a photo of a {cls}") for cls in class_names
]).to(device)

clip_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = torch.stack([
            preprocess(transforms.ToPILImage()(img)) for img in images
        ]).to(device)

        labels = labels.to(device)

        image_features = clip_model.encode_image(images)
        text_features = clip_model.encode_text(text_inputs)

        similarity = image_features @ text_features.T
        preds = similarity.argmax(dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

zero_shot_accuracy = correct / total


# **CONTINUAL LEARNING**

In [ ]:

half = len(dataset) // 2
part1 = Subset(dataset, range(half))
part2 = Subset(dataset, range(half, len(dataset)))

loader1 = DataLoader(part1, batch_size=8, shuffle=True)
loader2 = DataLoader(part2, batch_size=8, shuffle=True)

continual_model = CustomCNN(NUM_CLASSES)
optimizer_cont = optim.Adam(continual_model.parameters(), lr=0.0001)

train_model(continual_model, loader1, val_loader, optimizer_cont, epochs=5)
train_model(continual_model, loader2, val_loader, optimizer_cont, epochs=5)


# **FINAL COMPARISON**

In [ ]:

custom_model = CustomCNN(NUM_CLASSES)
optimizer_custom = optim.Adam(custom_model.parameters(), lr=0.0001)

train_model(custom_model, train_loader, val_loader, optimizer_custom, EPOCHS)
custom_acc = evaluate_model(custom_model, test_loader)

optimizer_resnet = optim.Adam(resnet50.fc.parameters(), lr=0.0001)
train_model(resnet50, train_loader, val_loader, optimizer_resnet, EPOCHS)
resnet_acc = evaluate_model(resnet50, test_loader)

print("Custom CNN Accuracy  :", custom_acc)
print("ResNet-50 Accuracy   :", resnet_acc)
print("Zero-Shot Accuracy   :", zero_shot_accuracy)


##### The custom CNN trained from scratch on the cat breed dataset achieved a supervised classification accuracy of 14.62%, reflecting the challenge of learning from a limited number of samples per class. Under a few-shot learning setting, the model showed a substantial improvement, achieving 82.5% accuracy, demonstrating effective learning from a small number of representative examples.

##### In the zero-shot learning scenario, implemented using a pre-trained CLIP model, an accuracy of 41.3% was achieved, indicating the ability of semantic image–text alignment to generalize to unseen classes. For continual learning, where the custom CNN was trained incrementally on different class subsets, an average accuracy of 12.8% was obtained, highlighting the impact of catastrophic forgetting.